# House prices advanced regression techniques

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, zscore
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.utils import shuffle

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV

sns.set_theme()

In [342]:
def get_nan_dummy(series):
    """Given a Series containing NaN and several classes return a dummy Series
    indicating 0 for NaN and 1 for non-NaN data
    
    Parameters
    ----------
    - series : pd.Series, input series or col to dummify
    
    Return
    ------
    - s : pd.Series the dummy Series
    """
    s = series.notna().astype(int).astype("category")
    s.name = f"{series.name}_abs_pres"
    return s

In [343]:
data = pd.read_csv("./train.csv")

# dropping the Id column which isn't informative
data = data.drop("Id", axis=1)

# Put SalePrice as first column
new_cols = data.columns.to_list()
new_cols.reverse()
data = data[new_cols]


# identifying variables with NaN
all_nan = data.isnull().sum().sort_values(ascending=False)
all_nan = all_nan[all_nan > 0]

# looking at the content of all_nan, we can assume we need the following variables
# to have a presence/absence aditional dummy variable
# Garage, Basement, Pool, Alley, Fence, Fireplace, Front lot, Masonry veneer
cols = ["PoolQC", "Alley", "Fence", "FireplaceQu", "LotFrontage", "MasVnrArea",
        "GarageQual", "BsmtQual"]
abs_pres_series = []

for col in cols:
    abs_pres_series.append(get_nan_dummy(data[col]))
    
abs_pres_series = pd.concat(abs_pres_series, axis=1)
data = pd.concat([data, abs_pres_series], axis=1) # we choose to either add these columns or not later

# There is a difference of NaN for Bsmt variables, let's look closer
a = data["BsmtExposure"].isna()[data["BsmtExposure"].isna() == True].index # 38 NaN
b = data["BsmtCond"].isna()[data["BsmtCond"].isna() == True].index # 37 NaN
mystery_row = [x for x in (set(a) - set(b))] 
mystery_row = mystery_row[0] # because there is only 1 element
data.loc[mystery_row,["BsmtExposure", "BsmtFinType2", "BsmtCond", "BsmtQual"]]

# we can either drop the data or assume BsmtExposure=No bc unfinished
# lets drop it
data.drop(mystery_row, axis=0, inplace=True)

In [344]:
# now we will fill the NaN of each values accordingly
# for qualitative data, Na suggest an absence rather than a lake of observation
#    so we assume that absence is actually an observation for some variables
#    we name them "abs" for abscence
data["PoolQC"].fillna("abs", inplace=True)
data["MiscFeature"].fillna("abs", inplace=True)
data["Alley"].fillna("abs", inplace=True)
data["Fence"].fillna("abs", inplace=True)
data["FireplaceQu"].fillna("abs", inplace=True)
data["MasVnrType"].fillna("abs", inplace=True)
data["MasVnrArea"].fillna(0, inplace=True)
data["Electrical"].fillna("abs", inplace=True)

#data["LotFrontage"] ? since there is no value of 0 in the column
# we can assume NaN -> 0
data["LotFrontage"].fillna(0, inplace=True)

# We could drop the NaN Bsmt and Garage rows, we leave it as an option for the time being only keeping the indexs
#data.dropna(subset=["GarageQual", "BsmtExposure"], inplace=True)
na_indexes = np.unique(np.where(data[['GarageQual', 'BsmtExposure']].isna() == True)[0]).tolist()

garage_mean = data["GarageYrBlt"].mean()
data["GarageYrBlt"].fillna(garage_mean, inplace=True)
data["GarageCond"].fillna("abs", inplace=True)
data["GarageType"].fillna("abs", inplace=True)
data["GarageFinish"].fillna("abs", inplace=True)
data["GarageQual"].fillna("abs", inplace=True)
data["BsmtExposure"].fillna("abs", inplace=True)
data["BsmtFinType2"].fillna("abs", inplace=True)
data["BsmtCond"].fillna("abs", inplace=True)
data["BsmtQual"].fillna("abs", inplace=True)
data["BsmtFinType1"].fillna("abs", inplace=True)
# for MasVnrArea, MasVnrType and Electrical, considering the few NA we choose
#    to drop it
data.dropna(subset=["MasVnrArea", "MasVnrType", "Electrical"], inplace=True)

# Changing CentralAir to one-hot
data["CentralAir"] = data["CentralAir"].replace({'Y': 1, 'N': 0}).astype("uint8")

# Let's look at the numerical data
data.select_dtypes(include=np.number)

# we have to change MSSubClass to categories
data["MSSubClass"] = data["MSSubClass"].astype("category")

# we now have 36 numerical variables, let's select them
cols = data.select_dtypes(include=np.number).columns.tolist()

# selecting the continuous variables, candidate for log transform
continuous = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1",
              "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF",
              "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea",
              "WoodDeckSF", "OpenPorchSF", "PoolArea", "SalePrice"]

# Doing the log transform, to do so we switch 0 -> NaN then from Nan -> 0 back again
data[continuous] = np.log(data[continuous].replace(0, np.nan)) # log transform, switching to nan to ingore the 0
data[continuous] = data[continuous].replace(np.nan, 0) # replacing nan by 0 again

# select categorical data
categoricals = data.select_dtypes(include="object").columns.to_list()

data[categoricals] = data[categoricals].astype("category")
enc = OrdinalEncoder()
data[categoricals] = pd.DataFrame(enc.fit_transform(data[categoricals]),
                                   columns=categoricals, index=data.index)

#data.to_csv("./clean_train.csv")

In [346]:
# Outlayers selection and dropping
## calculating the z-scores for values than are non zeros i.e excluding missing data
data[continuous] = data[continuous].replace(0, np.nan)
z_scores = []
for col in continuous:
    z_scores.append(zscore(data[continuous][col].dropna()).sort_values(ascending=False))
    
data[continuous] = data[continuous].replace(np.nan, 0) # setting back again NaN -> 0

## selecting outlayers indexes
threshold = 3.50 # z score limits
outlayers_list = []

for i in range(len(z_scores)):
    outlayers_list.extend(z_scores[i][z_scores[i] > threshold].index.tolist()) # select all outlayers > z
    outlayers_list.extend(z_scores[i][z_scores[i] < -threshold].index.tolist()) # select all outlayers < -z

outlayers_list = [x for x in set(outlayers_list)] # remove multiple occurences

## Dropping outlayers
print(data.shape)
data.drop(inplace=True, index=outlayers_list)
print(data.shape)

# Saving data without outlayers
#data.to_csv("./clean_train.csv", index=False)
## Dropping houses with no basements or no Garage
#data.drop(na_indexes, errors='ignore')

(1459, 88)
(1409, 88)


In [347]:
# GradientBoostingRegressor model

In [348]:
# Splitting input and target variables
X = data.iloc[:,1:]
y = data.iloc[:,0]

# shuffled data for cross val
X_shuf, y_shuf = shuffle(X, y, random_state=0)

# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [349]:
# # making a pipepline with ordinal encoder
# ordinal_encoder = make_column_transformer(
#     (
#         OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
#         make_column_selector(dtype_include="category"),
#     ),
#     remainder="passthrough",
#     # Use short feature names to make it easier to specify the categorical
#     # variables in the HistGradientBoostingRegressor in the next step
#     # of the pipeline.
#     verbose_feature_names_out=False,
# )

# reg_pipe = make_pipeline(
#     ordinal_encoder,
#     HistGradientBoostingRegressor(
#         random_state=0,
#         categorical_features=categoricals,
#     ),
# ).set_output(transform="pandas")

# #testing the pipeline on a 5fold CV
# #results = cross_validate(reg_pipe, X, y, cv=5)

# # Hyperparameter tunning

# param_grid = {"histgradientboostingregressor__learning_rate": [0.01, 0.05, 0.1, 0.2, 0.4, 0.7, 1],
#                "histgradientboostingregressor__max_iter": [100, 200, 400, 600, 1000, 1400, 1800, 2500],
#                "histgradientboostingregressor__max_leaf_nodes": [2, 4, 6, 10, 20, 40, 100, None],
#                "histgradientboostingregressor__max_depth": [1, 2, 5, 10, 20, 30, 40, 50, None],
#                "histgradientboostingregressor__min_samples_leaf": [1, 2, 3, 4, 5, 8, 10, 20]}

# search = HalvingRandomSearchCV(reg_pipe, param_distributions=param_grid,
#                                max_resources=10,
#                                random_state=0).fit(X, y)

# optimal_d = {}
# #removing 'histgradientboostingregressor__' in front of param
# rem = len('histgradientboostingregressor__')
# for (key, value) in search.best_params_.items():
#     optimal_d[key[rem:]] = value

# optimal_d = {'min_samples_leaf': 5,
#  'max_leaf_nodes': 40,
#  'max_iter': 400,
#  'max_depth': 30,
#  'learning_rate': 0.01,
#  'categorical_features': categoricals}
    
# reg_pipe_opt = make_pipeline(
#     ordinal_encoder,
#     HistGradientBoostingRegressor(**optimal_d),
# ).set_output(transform="pandas")

# results = cross_validate(reg_pipe_opt, X, y, cv=5)
# results['test_score'].mean()

In [350]:
# reg_pipe = make_pipeline(
#     ordinal_encoder,
#     HistGradientBoostingRegressor(
#         random_state=0,
#         categorical_features=categoricals,
#     ),
# ).set_output(transform="pandas")

# results = cross_validate(reg_pipe, X, y, cv=5)
# results['test_score'].mean()

In [351]:
reg = GradientBoostingRegressor()

# random_grid = {'n_estimators' : [100, 200, 400, 600, 800, 1200, 1600],
#                'learning_rate' : [0.01, 0.1, 0.2, 0.4, 0.6, 1, 1.5, 2],
#                'min_samples_split' : [2, 3, 4, 6, 10],
#                'min_samples_leaf' : [1, 2, 3, 4, 5, 8, 10],
#                'max_depth' : [1, 2, 5, 10, 20, 30, 40, 50, 80, 100],
#                'min_samples_split' : [2, 4, 8],
#                'min_samples_leaf' : [1, 2, 4, 6, 8, 10, 15],
#                'max_leaf_nodes' : [2, 4, 6, 10, 20, 40],}

#search = RandomizedSearchCV(estimator = reg, param_distributions = random_grid,
#                               n_iter = 200, cv = 3, verbose=1, random_state=0, n_jobs = -1)

#search.fit(X,y)

#saving best hyperparameter set as a dict
#best_d = search.best_params_

best_d = {'n_estimators': 1600,
          'min_samples_split': 4,
          'min_samples_leaf': 15,
          'max_leaf_nodes': 2,
          'max_depth': 10,
          'learning_rate': 0.1}

reg = GradientBoostingRegressor(**best_d)

score = cross_val_score(
             reg, X_shuf, y_shuf, cv=5)

print(f"{score.mean():.2%}")

reg.fit(X, y)

91.36%


GradientBoostingRegressor(max_depth=10, max_leaf_nodes=2, min_samples_leaf=15,
                          min_samples_split=4, n_estimators=1600)

In [ ]:
# with z thres = 3.6 and no Na deletion : 90.10% mean CV score
# with z thres = 3.6 and Na deletion : 90.10% mean CV score

# with z thres = 4 and no Na deletion : 89.62% mean CV score
# with z thres = 4 and Na deletion : 89.62% mean CV score

# with z thres = 3.4 and no Na deletion : 91.03% mean CV score
# with z thres = 3.4 and no Na deletion and n_estimators=1800 : 91.06% mean CV score
# with z thres = 3.4 and Na deletion and n_estimators=1800 : 91.06% mean CV score

# with z thres = 3.5 and no Na deletion and n_estimators=1800 : 91.28% mean CV score

# with z thres = 3.2 and no Na deletion and n_estimators=1800 : 90.84% mean CV score

# with z thres = 3.5 no Na deletion, best param and NO abs/pres columns :  90.80 %
# with z thres = 3.5 no Na deletion, best param and abs/pres columns : 91.21%

In [ ]:
# note for later : maybe remove outlayers after a test/train split set to evaluate
# the impact of extreme value drop on generalisation abilities of the model.

In [ ]:
# Outlayers selection and dropping
## calculating the z-scores for values than are non zeros i.e excluding missing data
X_train[continuous] = X_train[continuous].replace(0, np.nan)
z_scores = []
for col in continuous:
    z_scores.append(zscore(X_train[continuous][col].dropna()).sort_values(ascending=False))
    
X_train[continuous] = X_train[continuous].replace(np.nan, 0) # setting back again NaN -> 0

## selecting outlayers indexes
threshold = 3.55 # z score limits
outlayers_list = []

for i in range(len(z_scores)):
    outlayers_list.extend(z_scores[i][z_scores[i] > threshold].index.tolist()) # select all outlayers > z
    outlayers_list.extend(z_scores[i][z_scores[i] < -threshold].index.tolist()) # select all outlayers < -z

outlayers_list = [x for x in set(outlayers_list)] # remove multiple occurences

## Dropping outlayers
print(X_train.shape)
X_train.drop(inplace=True, index=outlayers_list)
y_train.drop(inplace=True, index=outlayers_list)
print(X_train.shape)

## Dropping houses with no basements or no Garage
#data.drop(na_indexes, errors='ignore')

In [ ]:
# Loading the data
data = pd.read_csv("./test.csv")

# dropping the Id column which isn't informative
data = data.drop("Id", axis=1)

# Put SalePrice as first column
new_cols = data.columns.to_list()
new_cols.reverse()
data = data[new_cols]

# looking at the content of all_nan, we can assume we need the following variables
# to have a presence/absence aditional dummy variable
# Garage, Basement, Pool, Alley, Fence, Fireplace, Front lot, Masonry veneer
cols = ["PoolQC", "Alley", "Fence", "FireplaceQu", "LotFrontage", "MasVnrArea",
        "GarageQual", "BsmtQual"]
abs_pres_series = []

for col in cols:
    abs_pres_series.append(get_nan_dummy(data[col]))
    
abs_pres_series = pd.concat(abs_pres_series, axis=1)
data = pd.concat([data, abs_pres_series], axis=1) # we choose to either add these columns or not later

# now we will fill the NaN of each values accordingly
# for qualitative data, Na suggest an absence rather than a lake of observation
#    so we assume that absence is actually an observation for some variables
#    we name them "abs" for abscence
data["PoolQC"].fillna("abs", inplace=True)
data["MiscFeature"].fillna("abs", inplace=True)
data["Alley"].fillna("abs", inplace=True)
data["Fence"].fillna("abs", inplace=True)
data["FireplaceQu"].fillna("abs", inplace=True)
data["MasVnrType"].fillna("abs", inplace=True)
data["MasVnrArea"].fillna(0, inplace=True)
data["Electrical"].fillna("abs", inplace=True)

#data["LotFrontage"] ? since there is no value of 0 in the column
# we can assume NaN -> 0
data["LotFrontage"].fillna(0, inplace=True)

# We could drop the NaN Bsmt and Garage rows, we leave it as an option for the time being only keeping the indexs
#data.dropna(subset=["GarageQual", "BsmtExposure"], inplace=True)
na_indexes = np.unique(np.where(data[['GarageQual', 'BsmtExposure']].isna() == True)[0]).tolist()

garage_mean = data["GarageYrBlt"].mean()
data["GarageYrBlt"].fillna(garage_mean, inplace=True)
data["GarageCond"].fillna("abs", inplace=True)
data["GarageType"].fillna("abs", inplace=True)
data["GarageFinish"].fillna("abs", inplace=True)
data["GarageQual"].fillna("abs", inplace=True)
data["BsmtExposure"].fillna("abs", inplace=True)
data["BsmtFinType2"].fillna("abs", inplace=True)
data["BsmtCond"].fillna("abs", inplace=True)
data["BsmtQual"].fillna("abs", inplace=True)
data["BsmtFinType1"].fillna("abs", inplace=True)
# for MasVnrArea, MasVnrType and Electrical, considering the few NA we choose
#    to drop it
#data.dropna(subset=["MasVnrArea", "MasVnrType", "Electrical"], inplace=True)

# Changing CentralAir to one-hot
data["CentralAir"] = data["CentralAir"].replace({'Y': 1, 'N': 0}).astype("uint8")

# Let's look at the numerical data
data.select_dtypes(include=np.number)

# we have to change MSSubClass to categories
data["MSSubClass"] = data["MSSubClass"].astype("category")

# we now have 36 numerical variables, let's select them
cols = data.select_dtypes(include=np.number).columns.tolist()

# selecting the continuous variables, candidate for log transform
continuous = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1",
              "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF",
              "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea",
              "WoodDeckSF", "OpenPorchSF", "PoolArea"]

# Doing the log transform, to do so we switch 0 -> NaN then from Nan -> 0 back again
data[continuous] = np.log(data[continuous].replace(0, np.nan)) # log transform, switching to nan to ingore the 0
data[continuous] = data[continuous].replace(np.nan, 0) # replacing nan by 0 again

# select categorical data
categoricals = data.select_dtypes(include="object").columns.to_list()

data[categoricals] = data[categoricals].astype("category")
enc = OrdinalEncoder()
data[categoricals] = pd.DataFrame(enc.fit_transform(data[categoricals]),
                                   columns=categoricals, index=data.index)

data.to_csv("./clean_test.csv", index=False)

In [ ]:
# data["GarageQual_abs_pres"][1116] # indicates there is no garage so GarageCars = 0
data.loc[1116,"GarageCars"] = 0

In [357]:
# For the other data we are going to create a serie of classifiers to infer the missing informations
# Since Saleprice is missing for the rows in which we try to infer variables
# we will combine the train and test data, removing SalePrice from the train data, in order to have
# more observation to train our classifiers
df_train = pd.read_csv("./clean_train.csv")
df_train.drop("SalePrice", axis=1, inplace=True)
df_test = pd.read_csv("./clean_test.csv")


df = pd.concat([df_train, df_test], axis = 0, ignore_index=True)
df_train = df.dropna() # dropping NaNs which are the also rows to infer

In [ ]:
# There are missing values in the data, we need to take a closer look
all_nan = df_test.isna().sum().sort_values(ascending=False)
all_nan = all_nan[all_nan > 0]

# Now lets get all the indexes of values to predict
to_predict = {}
for col in all_nan.index.to_list():
    to_predict[col] = (df_test[col][df_test[col].isna() == True]).index.to_list()

In [ ]:
def build_classifier(data, col):
    """Build a classifier for the variable specified in col
    
    Parameters
    ----------
     - data, pd.Dataframe : The data used to train the classifier
     - col, string : Column name of the variable to infer, must be in data
     
    Returns
    -------
     - clf, HistGradientBoostingClassifier() : trained classifier
     """   
    X = data.loc[:, data.columns != col]
    y = data[col]

    clf = HistGradientBoostingClassifier()

    random_grid = {"learning_rate": [0.01, 0.05, 0.1, 0.2, 0.4, 0.7, 1],
                   "max_iter": [100, 200, 400, 600, 1000, 1400, 1800, 2500],
                   "max_leaf_nodes": [2, 4, 6, 10, 20, 40, 100],
                   "max_depth": [1, 2, 5, 10, 20, 30, 40, 50],
                   "min_samples_leaf": [1, 2, 3, 4, 5, 8, 10, 20]}

    search = RandomizedSearchCV(estimator = clf, param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=1, random_state=0, n_jobs = -1)
    search.fit(X,y)
    best_d = search.best_params_
    
    clf = HistGradientBoostingClassifier(**best_d)
    clf.fit(X,y)
    
    return clf
    
    

In [ ]:
# Filled NA with predictors
# for col in to_predict:
#     clf = build_classifier(df_train, col)
#     print(col)
#     print(df_test.loc[to_predict[col], col])
#     df_test.loc[to_predict[col], col] = clf.predict(df_test.drop(col, axis=1).loc[to_predict[col],:])
#     print(df_test.loc[to_predict[col], col])

# df_test.to_csv("./clean_nona_test.csv", index=False)

In [358]:
#df_test = pd.read_csv("./clean_nona_test.csv")

In [359]:
#df_test[categoricals] = df_test[categoricals].astype("category")

In [367]:
#y_pred = reg.predict(df_test)
#y_pred = pd.Series(np.exp(y_pred), name="SalePrice")

In [369]:
#original_df = pd.read_csv("./test.csv")
#res = pd.concat([original_df["Id"], y_pred], names=["Id", "SalePrice"], axis=1)
#res.to_csv("resultsGBM.csv", index=False)

In [2]:
df_train = pd.read_csv("./clean_train.csv")